<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/%EC%9E%AC%EB%AC%B4%EC%A0%95%EB%B3%B4_%EC%B6%94%EA%B0%80_%EB%AA%A8%EB%93%88_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install finance-datareader
!pip install opendartreader
!pip install --upgrade opendartreader
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

# 재무재표 로딩
# import OpenDartReader


In [8]:
def money_sur(stock_name, start, end):

  # 라이브러리를 찾음... 한방에 나옴...
    start_date = start
    end_date = end
    
    if type(stock_name) != str:
        sample_code = str(stock_name).zfill(6)  
    else:
        sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [9]:
end_date = datetime.datetime.now().strftime("%Y%m%d")

df = money_sur(660, '20180101', end_date)

In [10]:
df

,Date,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
0,2018-01-02,77300,77300,76200,76600,2014838,0.001307,32990,18.31,2.32,4184,0.78,600
1,2018-01-03,78100,78800,77300,77700,3097644,0.014360,32990,18.57,2.36,4184,0.77,600
2,2018-01-04,78700,79700,77100,77100,3729328,-0.007722,32990,18.43,2.34,4184,0.78,600
3,2018-01-05,78400,79400,78300,79300,3395065,0.028534,32990,18.95,2.40,4184,0.76,600
4,2018-01-08,79500,80000,77400,78200,2960495,-0.013871,32990,18.69,2.37,4184,0.77,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,2021-09-09,105000,106000,103000,103000,3233011,-0.028302,71275,14.82,1.45,6952,1.14,1170
911,2021-09-10,103500,105500,103000,105000,1999369,0.019417,71275,15.10,1.47,6952,1.11,1170
912,2021-09-13,105000,107000,104000,106500,1923344,0.014286,71275,15.32,1.49,6952,1.10,1170
913,2021-09-14,107500,108500,106500,107500,2463248,0.009390,71275,15.46,1.51,6952,1.09,1170


=========== 오류 코드 수정 9.13,,2017~2021 실행 가능 확인[ 업종 코드 = 5380 5930 51910,660 ] 

In [ ]:
### 사용 안해도 됨...
### 총수입정도 알수 있을 듯 싶으나 쓸모없음 --- 폐기

def pro_equ(code, start, end):
    # OpenDartReader 사용
    api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
    dart = OpenDartReader(api_key)
    arr = [] # 재무 정보
    index = [] # 계산지표
    
    # 주식정보
    if type(code) != str:
        code = str(code).zfill(6)       
    
    if type(start) != str:
        start = str(start)
    
    if type(end) != str:
        end = str(end)
        
    stock = fdr.DataReader(code, start = str(int(start)+1), end = end)  # 2021 지표 <- 2020,2019 데이터로 만듬
    stock = stock.reset_index()
    stock['PER'] = 0.1
    stock['PBR'] = 0.1
    stock['ROE'] = 0.1
    stock['ROA'] = 0.1
    
    #         ------------------------
    
    small = dart.report('005930', '소액주주', int(datetime.datetime.now().strftime("%Y"))-1, reprt_code=11014)
    stock_tot_co = int(small['stock_tot_co'].str.replace(',', '')) # 총 발행 주식 수
    
    for i in range(int(start[:4]), int(end[:4])):
        # https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016 --> 입력변수 명 참고
        # corp: 종목코드, bsns_year: 사업연도, reprt_code: 보고서코드
        # (1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011)
        fs_before = dart.finstate_all(corp = code, bsns_year= str(i-1), fs_div='CFS', reprt_code=11011) 
        fs_present = dart.finstate_all(corp = code, bsns_year= i, fs_div='CFS', reprt_code=11014)

        # 자본, 부채는 당기 금액('thstrm_amount') 값을 통해서 구함
#         equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount'])
#         print(equity)
#         liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))
#         print(liability)
        # 자산 총계
        assets = equity + liability
        
        # <예시>
        # 2019 4분기 ~ 2020 3분기까지의 당기순이익의 합을 구하려면 2019년 4분기 당기순이익과 2020년 1분기 ~ 3분기 당기순이익의 합을 알아여함
        # 2020년 1분기 ~ 3분기 당기순이익의 합은 2020년 3분기 손익계산서에서 'thstrm_add_amount' 값을 가져오면 되고
        # 2019년 4분기 당기순이익은 2019년 전체 당기순이익에서 2019년 1분기 ~ 3분기 당기순이익의 합을 빼서 구할 수 있음
        
        # frmtrm_add_amount --> 전기 금액, thstrm_amount ---> 당기 금액, thstrm_add_amount --> 당기 누적금액
#         profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "")) # 당기순이익
        
        pf_l = ['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)','순이익(손실)','분기순이익(손실)','연결분기순이익','연결당기순이익', '연결분기순이익(손실)']
        for i in range(len(pf_l)):
            if pf_l[i] in list(fs_present['account_nm'].unique()):
                pf = pf_l[i]
            if pf_l[i] in list(fs_before['account_nm'].unique()):
                pf_b = pf_l[i]
        
        sj_l = ['IS','CIS']
        for i in range(len(sj_l)):
            if sj_l[i] in list(fs_present['sj_div'].unique()):
                sj = sj_l[i]
            if sj_l[i] in list(fs_before['sj_div'].unique()):
                sj_b = sj_l[i]
        
#         print(pf)
        profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin([sj]) & fs_present['account_nm'].isin([pf]), 'frmtrm_add_amount'].iloc[0].replace(",", ""))
#         print(profit_before_3Q)
#         profit_before = int(fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")) # 당기순이익
        profit_before = int(fs_before.loc[fs_before['sj_div'].isin([sj_b]) & fs_before['account_nm'].isin([pf_b]), 'thstrm_amount'].iloc[0].replace(",", ""))
#         print(profit_before)
#         profit_present = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
        profit_present = int(fs_present.loc[fs_present['sj_div'].isin([sj]) & fs_present['account_nm'].isin([pf]), 'thstrm_add_amount'].iloc[0].replace(",", "")) # 당기순이익
#         print(profit_present)
        profit = (profit_before-profit_before_3Q) + profit_present
        
        a = (equity, liability, assets, profit, stock_tot_co) 
        # 자본총계(당기자본), 부채총계(당기부채), 자산총계, 당기 순이익, 총 발행 주식수
        arr.append(a)
        
#         - PER(주가 수익률) : 주가를 1주당 순이익(EPS)로 나눈 값
#         - PBR(주가 순자산 비율) : 주가를 1주당 순자산가치로 나눈 값
#         - ROE(자기자본이익률) : 당기순이익을 순자산(자기자본)으로 나눈 값
#         - ROA(총자산수익률) : 당기순이익을 자산총액으로 나눈 값
        
#         print(stock[stock['Date'].dt.year == i]['Close'])
            
        
        ROA = profit/assets
        stock.loc[stock['Date'].dt.year == i+1,'ROA'] = ROA
        
        EPS = profit/stock_tot_co
        PER = stock.loc[stock['Date'].dt.year == i+1,'Close']/EPS
        
#         PER = stock[stock['Date'].dt.year == i+1]['Close']/EPS
        
        BPS = equity/stock_tot_co
        PBR = stock.loc[stock['Date'].dt.year == i+1, 'Close']/BPS
#         PBR = stock[stock['Date'].dt.year == i+1]['Close']/BPS
        
        for i in PER.index:
            stock['PER'][i] = PER[i]
            stock['PBR'][i] = PBR[i]
        
        b = (EPS,BPS)
        # EPS: 순이익/총발행주식수 --> 주당 수익?
        # BPS: 순자산/총발행 주식수 --> 주당 자산가치?
        
        index.append(b)
        
    stock['ROE'] = stock['PBR']/stock['PER']
    
    return arr,index, stock

In [ ]:
end_date = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
arr, indexs, df = pro_equ(660,2017,end_date)  # 2018의 경우 2019,2020,2021까지의 결과 도출

30904498000000
9825955000000
당기순이익
43488826000000
13888426000000
당기순이익
48266226000000
15405525000000
분기순이익
50499059000000
18768114000000
분기순이익


/////////////////

In [ ]:
api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
dart = OpenDartReader(api_key)

In [ ]:
stock = fdr.DataReader('000660', start = '20180101', end = end_date)

In [ ]:
fs_present3 = dart.finstate_all(corp = '005380', bsns_year= '2017', fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present2 = dart.finstate_all(corp = '005380', bsns_year= '2016', fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present2 = dart.finstate_all(corp = '005380', bsns_year= '2020', fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present3[fs_present3['sj_div'].isin(['IS','CIS']) & fs_present3['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
56,20180402003684,11011,2017,00356361,IS,손익계산서,ifrs_ProfitLoss,당기순이익(손실),-,제 17 기,2021973000000,제 16 기,1280994000000,제 15 기,1148531000000,12,
61,20180402003684,11011,2017,00356361,CIS,포괄손익계산서,ifrs_ProfitLoss,당기순이익(손실),-,제 17 기,2021973000000,제 16 기,1280994000000,제 15 기,1148531000000,0,


In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS']) & fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount


In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
56,20201116001588,11014,2020,00164742,IS,손익계산서,ifrs-full_Revenue,매출액,-,제 53 기 3분기,27575812000000,NaN,NaN,0,74754251000000,제 52 기 3분기,26968853000000,77922285000000
57,20201116001588,11014,2020,00164742,IS,손익계산서,ifrs-full_CostOfSales,매출원가,-,제 53 기 3분기,22450545000000,NaN,NaN,1,61651847000000,제 52 기 3분기,22540467000000,64958197000000
58,20201116001588,11014,2020,00164742,IS,손익계산서,ifrs-full_GrossProfit,매출총이익,-,제 53 기 3분기,5125267000000,NaN,NaN,2,13102404000000,제 52 기 3분기,4428386000000,12964088000000
59,20201116001588,11014,2020,00164742,IS,손익계산서,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,-,제 53 기 3분기,5439090000000,NaN,NaN,3,11962130000000,제 52 기 3분기,4049866000000,10522972000000
60,20201116001588,11014,2020,00164742,IS,손익계산서,dart_OperatingIncomeLoss,영업이익(손실),-,제 53 기 3분기,-313823000000,NaN,NaN,4,1140274000000,제 52 기 3분기,378520000000,2441116000000
61,20201116001588,11014,2020,00164742,IS,손익계산서,-표준계정코드 미사용-,공동기업및관계기업투자손익,-,제 53 기 3분기,11103000000,NaN,NaN,5,45812000000,제 52 기 3분기,74735000000,442635000000
62,20201116001588,11014,2020,00164742,IS,손익계산서,ifrs-full_FinanceIncome,금융수익,-,제 53 기 3분기,239912000000,NaN,NaN,6,614676000000,제 52 기 3분기,177833000000,611444000000
63,20201116001588,11014,2020,00164742,IS,손익계산서,ifrs-full_FinanceCosts,금융비용,-,제 53 기 3분기,202767000000,NaN,NaN,7,689154000000,제 52 기 3분기,130808000000,396687000000
64,20201116001588,11014,2020,00164742,IS,손익계산서,dart_OtherGains,기타수익,-,제 53 기 3분기,322340000000,NaN,NaN,8,919590000000,제 52 기 3분기,362966000000,979346000000
65,20201116001588,11014,2020,00164742,IS,손익계산서,dart_OtherLosses,기타비용,-,제 53 기 3분기,419066000000,NaN,NaN,9,1072965000000,제 52 기 3분기,434230000000,1046051000000


In [ ]:
fs_present2[fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount


In [ ]:
fs_present2.loc[fs_present2['sj_div'].isin(['IS','CIS']) & fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익']), 'frmtrm_add_amount'].replace(",", "")

55    1011037000000
60    1011037000000
Name: frmtrm_add_amount, dtype: object

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익']), 'frmtrm_add_amount'].replace(",", "")

60    12142065000000
Name: frmtrm_add_amount, dtype: object

In [ ]:
fs_present = dart.finstate_all(corp = '000660', bsns_year= '2019', fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
55,20181114001530,11014,2018,00126380,IS,손익계산서,ifrs_Revenue,수익(매출액),-,제 50 기 3분기,65459993000000,NaN,NaN,0,184506365000000,제 49 기 3분기,62048901000000,173596964000000
56,20181114001530,11014,2018,00126380,IS,손익계산서,ifrs_CostOfSales,매출원가,-,제 50 기 3분기,35194358000000,NaN,NaN,1,98378427000000,제 49 기 3분기,33004119000000,93559545000000
57,20181114001530,11014,2018,00126380,IS,손익계산서,ifrs_GrossProfit,매출총이익,-,제 50 기 3분기,30265635000000,NaN,NaN,2,86127938000000,제 49 기 3분기,29044782000000,80037419000000
58,20181114001530,11014,2018,00126380,IS,손익계산서,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,-,제 50 기 3분기,12690770000000,NaN,NaN,3,38041868000000,제 49 기 3분기,14511623000000,41539352000000
59,20181114001530,11014,2018,00126380,IS,손익계산서,dart_OperatingIncomeLoss,영업이익(손실),-,제 50 기 3분기,17574865000000,NaN,NaN,4,48086070000000,제 49 기 3분기,14533159000000,38498067000000
60,20181114001530,11014,2018,00126380,IS,손익계산서,dart_OtherGains,기타수익,-,제 50 기 3분기,176612000000,NaN,NaN,5,915176000000,제 49 기 3분기,375774000000,1036412000000
61,20181114001530,11014,2018,00126380,IS,손익계산서,dart_OtherLosses,기타비용,-,제 50 기 3분기,152354000000,NaN,NaN,6,620026000000,제 49 기 3분기,278122000000,778761000000
62,20181114001530,11014,2018,00126380,IS,손익계산서,-표준계정코드 미사용-,지분법이익,-,제 50 기 3분기,118455000000,NaN,NaN,7,289126000000,제 49 기 3분기,80423000000,122114000000
63,20181114001530,11014,2018,00126380,IS,손익계산서,ifrs_FinanceIncome,금융수익,-,제 50 기 3분기,2501153000000,NaN,NaN,8,7903413000000,제 49 기 3분기,2610849000000,6435175000000
64,20181114001530,11014,2018,00126380,IS,손익계산서,-표준계정코드 미사용-,금융비용,-,제 50 기 3분기,2249387000000,NaN,NaN,9,7021672000000,제 49 기 3분기,2407343000000,5821426000000


In [ ]:
16629519000000- 4487454000000 == 12142065000000

True

In [ ]:
fs_present[fs_present['sj_div'].isin(['IS','CIS'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
48,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,ifrs-full_Revenue,매출액,-,제 72 기 3분기,6838766000000,NaN,NaN,0,20063622000000,제 71 기 3분기,11416788000000,30506985000000
49,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,ifrs-full_CostOfSales,매출원가,-,제 72 기 3분기,5023127000000,NaN,NaN,1,13550971000000,제 71 기 3분기,3871624000000,11017081000000
50,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,ifrs-full_GrossProfit,매출총이익,-,제 72 기 3분기,1815639000000,NaN,NaN,2,6512651000000,제 71 기 3분기,7545164000000,19489904000000
51,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,-,제 72 기 3분기,1343078000000,NaN,NaN,3,4035970000000,제 71 기 3분기,1072744000000,3076229000000
52,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익,-,제 72 기 3분기,472561000000,NaN,NaN,4,2476681000000,제 71 기 3분기,6472420000000,16413675000000
53,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,ifrs-full_FinanceIncome,금융수익,-,제 72 기 3분기,416962000000,NaN,NaN,5,1305369000000,제 71 기 3분기,172835000000,1176190000000
54,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,ifrs-full_FinanceCosts,금융비용,-,제 72 기 3분기,389229000000,NaN,NaN,6,1113882000000,제 71 기 3분기,251999000000,929616000000
55,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,-표준계정코드 미사용-,지분법투자 관련 손익,-,제 72 기 3분기,4469000000,NaN,NaN,7,14522000000,제 71 기 3분기,2979000000,11451000000
56,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,dart_OtherGains,기타영업외수익,-,제 72 기 3분기,31440000000,NaN,NaN,8,80822000000,제 71 기 3분기,57491000000,74725000000
57,20191114002661,11014,2019,00164779,CIS,포괄손익계산서,dart_OtherLosses,기타영업외비용,-,제 72 기 3분기,15904000000,NaN,NaN,9,87652000000,제 71 기 3분기,18492000000,116906000000


In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익']), 'thstrm_amount'].replace(",", "")

Series([], Name: thstrm_amount, dtype: object)

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익']), 'frmtrm_add_amount']

56    1331909000000
Name: frmtrm_add_amount, dtype: object

In [ ]:
fs_present2[fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
69,20181114001530,11014,2018,00126380,IS,손익계산서,ifrs_ProfitLossAttributableToOwnersOfParent,지배기업의 소유주에게 귀속되는 당기순이익(손실),-,제 50 기 3분기,12967428000000,NaN,NaN,15,35560808000000,제 49 기 3분기,11039771000000,29328245000000


In [ ]:
fs_present.sj_div.value_counts()

BS     45
SCE    39
CF     32
CIS    24
Name: sj_div, dtype: int64

In [ ]:
fs_present[fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount


In [ ]:
fs_present[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount


In [ ]:
fs_present[fs_present['sj_div'].isin(['BS'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
0,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 71 기 3분기말,17700533000000,제 70 기말,17310444000000,1,NaN,NaN,NaN,NaN
1,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 71 기 3분기말,2340357000000,제 70 기말,2949991000000,2,NaN,NaN,NaN,NaN
2,20181114002438,11014,2018,00164779,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 71 기 3분기말,388229000000,제 70 기말,4674862000000,3,NaN,NaN,NaN,NaN
3,20181114002438,11014,2018,00164779,BS,재무상태표,dart_CurrentAvailableForSaleFinancialAssets,단기투자자산,-,제 71 기 3분기말,2946807000000,제 70 기말,929801000000,4,NaN,NaN,NaN,NaN
4,20181114002438,11014,2018,00164779,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 71 기 3분기말,7711347000000,제 70 기말,5552795000000,5,NaN,NaN,NaN,NaN
5,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_TradeAndOtherCurrentReceivables,기타수취채권,-,제 71 기 3분기말,16101000000,제 70 기말,37613000000,6,NaN,NaN,NaN,NaN
6,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_Inventories,재고자산,-,제 71 기 3분기말,3686740000000,제 70 기말,2640439000000,7,NaN,NaN,NaN,NaN
7,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_CurrentTaxAssets,당기법인세자산,-,제 71 기 3분기말,1663000000,제 70 기말,1305000000,8,NaN,NaN,NaN,NaN
8,20181114002438,11014,2018,00164779,BS,재무상태표,dart_OtherCurrentAssets,기타유동자산,-,제 71 기 3분기말,609289000000,제 70 기말,523638000000,9,NaN,NaN,NaN,NaN
9,20181114002438,11014,2018,00164779,BS,재무상태표,ifrs_NoncurrentAssets,비유동자산,-,제 71 기 3분기말,39676719000000,제 70 기말,28108020000000,10,NaN,NaN,NaN,NaN


In [ ]:
fs_present['account_nm'].value_counts()

기초자본             8
기말자본             8
당기순이익            5
확정급여제도의 재측정요소    4
환율변동             4
                ..
기타유동자산           1
자본금              1
비유동부채            1
기타영업외수익          1
기타수취채권의 증가       1
Name: account_nm, Length: 103, dtype: int64

In [ ]:
df

,Date,Open,High,Low,Close,Volume,Change,PER,PBR,ROE,ROA
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,8.403220,1.445614,0.172031,0.122212
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,8.502043,1.462615,0.172031,0.122212
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,8.413103,1.447314,0.172031,0.122212
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,8.584395,1.476782,0.172031,0.122212
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,8.567925,1.473949,0.172031,0.122212
...,...,...,...,...,...,...,...,...,...,...,...
907,2021-09-06,76800,77600,76600,77300,12861180,0.009138,18.552538,1.671147,0.090076,0.066190
908,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,18.264530,1.645204,0.090076,0.066190
909,2021-09-08,76000,76400,75600,76300,11798147,0.002628,18.312532,1.649528,0.090076,0.066190
910,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,18.072525,1.627909,0.090076,0.066190


In [ ]:
arr

[(210691251000000,
  85887328000000,
  296578579000000,
  36245426000000,
  5969782550),
 (242103160000000,
  95092626000000,
  337195786000000,
  47577132000000,
  5969782550),
 (263442244000000,
  89943741000000,
  353385985000000,
  24607128000000,
  5969782550),
 (276136188000000,
  99652554000000,
  375788742000000,
  24873372000000,
  5969782550)]

In [ ]:
indexs

[(6071.481782866614, 35292.952337099785),
 (7969.659129376496, 40554.77029058621),
 (4121.947121842822, 44129.28641764347),
 (4166.54573121763, 46255.65264517047)]

### ============= 연습 =============

In [ ]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 912 entries, 2018-01-02 to 2021-09-10
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    912 non-null    int64  
 1   High    912 non-null    int64  
 2   Low     912 non-null    int64  
 3   Close   912 non-null    int64  
 4   Volume  912 non-null    int64  
 5   Change  912 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 49.9 KB


In [ ]:
stock = stock.reset_index()

In [ ]:
pd.merge(stock,PER)

,Date,Open,High,Low,Close,Volume,Change
0,2020-12-09,72100,73900,72000,73900,24077996,0.030683
1,2020-12-23,72400,74000,72300,73900,19411326,0.022130
2,2021-08-18,73900,74600,73100,73900,29192631,-0.004043
3,2020-12-17,73400,73700,72600,73300,24293214,-0.006775
4,2021-08-23,73300,74000,73000,73300,19384648,0.008253
...,...,...,...,...,...,...,...
479,2021-09-03,76400,76700,76000,76600,12096419,0.007895
480,2021-09-06,76800,77600,76600,77300,12861180,0.009138
481,2021-09-07,77100,77100,75900,76100,13239401,-0.015524
482,2021-09-08,76000,76400,75600,76300,11798147,0.002628


In [ ]:
PER = stock[stock['Date'].dt.year == 2021]['Close']/10
for i in PER.index:
    stock['PER'][i] = PER[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
stock['per'] = 0.1

In [ ]:
a,b = 15,10

In [ ]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    668 non-null    datetime64[ns]
 1   Open    668 non-null    int64         
 2   High    668 non-null    int64         
 3   Low     668 non-null    int64         
 4   Close   668 non-null    int64         
 5   Volume  668 non-null    int64         
 6   Change  668 non-null    float64       
 7   per     668 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 41.9 KB


In [ ]:
stock.loc[stock['Date'].dt.year == 2020,'per'] = b

In [ ]:
stock

,Date,Open,High,Low,Close,Volume,Change,per
0,2019-01-02,39400,39400,38550,38750,7847664,0.001292,0.1
1,2019-01-03,38300,38550,37450,37600,12471493,-0.029677,0.1
2,2019-01-04,37450,37600,36850,37450,14108958,-0.003989,0.1
3,2019-01-07,38000,38900,37800,38750,12748997,0.034713,0.1
4,2019-01-08,38000,39200,37950,38100,12756554,-0.016774,0.1
...,...,...,...,...,...,...,...,...
663,2021-09-06,76800,77600,76600,77300,12861180,0.009138,15.0
664,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,15.0
665,2021-09-08,76000,76400,75600,76300,11798147,0.002628,15.0
666,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,15.0


In [ ]:
stock[stock['Date'].dt.year == 2020]

,Date,Open,High,Low,Close,Volume,Change,per
246,2020-01-02,55500,56000,55000,55200,12993228,-0.010753,10.0
247,2020-01-03,56000,56600,54900,55500,15422255,0.005435,10.0
248,2020-01-06,54900,55600,54600,55500,10278951,0.000000,10.0
249,2020-01-07,55700,56400,55600,55800,10009778,0.005405,10.0
250,2020-01-08,56200,57400,55900,56800,23501171,0.017921,10.0
...,...,...,...,...,...,...,...,...
489,2020-12-23,72400,74000,72300,73900,19411326,0.022130,10.0
490,2020-12-24,74100,78800,74000,77800,32502870,0.052774,10.0
491,2020-12-28,79000,80100,78200,78700,40085044,0.011568,10.0
492,2020-12-29,78800,78900,77300,78300,30339449,-0.005083,10.0


In [ ]:
PER[738]

8300.0

In [ ]:
stock['PER'][738]

83000.0

In [ ]:
stock['PER'][738] = PER[738]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
stock['PER'][738]

8300.0

In [ ]:
PER.index

738

In [ ]:
stock[stock['Date'].dt.year == 2021]['PER'] = PER

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
stock[stock['Date'].dt.year == 2021]['PER']

738   NaN
739   NaN
740   NaN
741   NaN
742   NaN
       ..
907   NaN
908   NaN
909   NaN
910   NaN
911   NaN
Name: PER, Length: 174, dtype: float64

In [ ]:
stock

,Date,Open,High,Low,Close,Volume,Change,PER
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,51020.0
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,51620.0
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,51080.0
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,52120.0
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,52020.0
...,...,...,...,...,...,...,...,...
907,2021-09-06,76800,77600,76600,77300,12861180,0.009138,NaN
908,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,NaN
909,2021-09-08,76000,76400,75600,76300,11798147,0.002628,NaN
910,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,NaN


In [ ]:
a =stock.copy()
a=a.reset_index()

In [ ]:
s = a[a['Date'].dt.year == 2018]['Close']

In [ ]:
a['aaaa'] = 0.1

In [ ]:
a.head()

,Date,Open,High,Low,Close,Volume,Change,aaaa
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,51020.0
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,51620.0
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,51080.0
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,52120.0
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,52020.0


In [ ]:
a['aaaa'] = s

In [ ]:
a.loc[a['Date'].dt.year == 2018,'Close']/1

0      51020.0
1      51620.0
2      51080.0
3      52120.0
4      52020.0
        ...   
239    38650.0
240    38800.0
241    38350.0
242    38250.0
243    38700.0
Name: Close, Length: 244, dtype: float64

In [ ]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

In [ ]:
dart.report('005930', '소액주주', 2020, reprt_code=11014)

,rcept_no,corp_cls,corp_code,corp_name,se,shrholdr_co,shrholdr_tot_co,shrholdr_rate,hold_stock_co,stock_tot_co,hold_stock_rate
0,20201116001248,Y,00126380,삼성전자,소액주주,"1,754,623","1,754,776",99.99%,"3,711,273,219","5,969,782,550",62.17%


In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount']

52    275948016000000
Name: thstrm_amount, dtype: object

=============코드 수정=======오류 파악============

In [ ]:
api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
dart = OpenDartReader(api_key)

In [ ]:
fs_before = dart.finstate_all(corp = '005930', bsns_year= '2020', fs_div='CFS', reprt_code=11011) 
fs_present = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount']

52    276136188000000
Name: thstrm_amount, dtype: object

In [ ]:
fs_present.iloc[52]

rcept_no               20201116001248
reprt_code                      11014
bsns_year                        2020
corp_code                    00126380
sj_div                             BS
sj_nm                           재무상태표
account_id           ifrs-full_Equity
account_nm                       자본총계
account_detail                      -
thstrm_nm                 제 52 기 3분기말
thstrm_amount         276136188000000
frmtrm_nm                     제 51 기말
frmtrm_amount         262880421000000
ord                                55
thstrm_add_amount                 NaN
frmtrm_q_nm                       NaN
frmtrm_q_amount                   NaN
frmtrm_add_amount                 NaN
Name: 52, dtype: object

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

42    99652554000000
Name: thstrm_amount, dtype: object

In [ ]:
fs_present.iloc[42]

rcept_no                    20201116001248
reprt_code                           11014
bsns_year                             2020
corp_code                         00126380
sj_div                                  BS
sj_nm                                재무상태표
account_id           ifrs-full_Liabilities
account_nm                            부채총계
account_detail                           -
thstrm_nm                      제 52 기 3분기말
thstrm_amount               99652554000000
frmtrm_nm                          제 51 기말
frmtrm_amount               89684076000000
ord                                     44
thstrm_add_amount                      NaN
frmtrm_q_nm                            NaN
frmtrm_q_amount                        NaN
frmtrm_add_amount                      NaN
Name: 42, dtype: object

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2019')

In [ ]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

44    89684076000000
Name: thstrm_amount, dtype: object

In [ ]:
frmtrm_add_amount

In [ ]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

68    26090846000000
Name: thstrm_amount, dtype: object

In [ ]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")

68    26090846000000
Name: thstrm_amount, dtype: object

In [ ]:
fs_before.iloc[68]

rcept_no                                               20210309000744
reprt_code                                                      11011
bsns_year                                                        2020
corp_code                                                    00126380
sj_div                                                             IS
sj_nm                                                           손익계산서
account_id           ifrs-full_ProfitLossAttributableToOwnersOfParent
account_nm                                 지배기업의 소유주에게 귀속되는 당기순이익(손실)
account_detail                                                      -
thstrm_nm                                                      제 52 기
thstrm_amount                                          26090846000000
frmtrm_nm                                                      제 51 기
frmtrm_amount                                          21505054000000
bfefrmtrm_nm                                                   제 50 기
bfefrmtrm_amount    

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2018',fs_div='CFS', reprt_code=11011)

In [ ]:
a.loc[a['sj_div'].isin(['IS']) & a['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

73    43890877000000
Name: thstrm_amount, dtype: object

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',
       'account_id', 'account_nm', 'account_detail', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'ord',
       'thstrm_add_amount', 'frmtrm_q_nm', 'frmtrm_q_amount',
       'frmtrm_add_amount'],
      dtype='object')

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "") # 당기순이익

68    16277059000000
Name: frmtrm_add_amount, dtype: object

In [ ]:
fs_present.iloc[68]

rcept_no                                               20201116001248
reprt_code                                                      11014
bsns_year                                                        2020
corp_code                                                    00126380
sj_div                                                             IS
sj_nm                                                           손익계산서
account_id           ifrs-full_ProfitLossAttributableToOwnersOfParent
account_nm                                 지배기업의 소유주에게 귀속되는 당기순이익(손실)
account_detail                                                      -
thstrm_nm                                                  제 52 기 3분기
thstrm_amount                                           9266814000000
frmtrm_nm                                                         NaN
frmtrm_amount                                                     NaN
ord                                                                15
thstrm_add_amount   

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")

68    19645377000000
Name: thstrm_add_amount, dtype: object

In [ ]:
fs_present['sj_div'].isin(['BS']).index

RangeIndex(start=0, stop=205, step=1)

In [ ]:
fs_present['account_id'].isin(['기말자본']).index

RangeIndex(start=0, stop=205, step=1)

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

102287702000000

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
52,20201116001248,11014,2020,00126380,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 52 기 3분기말,276136188000000,제 51 기말,262880421000000,55,NaN,NaN,NaN,NaN


In [ ]:
fs_2019 = dart.finstate_all(corp='005930', bsns_year='2019', fs_div='CFS', reprt_code=11011) 
fs_2020_3Q = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11014) 



In [ ]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", ""))


16277059000000

In [ ]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", ""))


19645377000000

In [ ]:
fs_present['account_id'].isin(['ifrs-full_Equity'])

0      False
1      False
2      False
3      False
4      False
       ...  
200    False
201    False
202    False
203    False
204    False
Name: account_id, Length: 205, dtype: bool